In [61]:
import json
import os
from os import path
from collections import defaultdict
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [62]:
train_set = "/home/shtoshni/Research/events/proc_data/kbp_2015/train.512.jsonlines"

In [63]:
span_width = defaultdict(int)
span_count_fracs = []
doc_lengths = []
num_bert_windows = []

with open(train_set) as f:
    for line in f:
        instance = json.loads(line.strip())
        clusters = instance["clusters"]
        doc_length = sum([len(sent) for sent in instance["sentences"]])
        doc_lengths.append(doc_length)
        num_bert_windows.append(len(instance["sentences"]))
        span_count_frac = 0.0
        for cluster in clusters:
            for (span_start, span_end, mention_info) in cluster:
                span_width[span_end - span_start + 1] += 1
                span_count_frac += 1.0/doc_length
                
        span_count_fracs.append(span_count_frac)

In [64]:
fig = make_subplots(rows=2, cols=2, 
                   subplot_titles=("Document Length Histogram", "# of BERT Windos in Doc", 
                                   "Span Width Bar Plot", "# of Mentions as a Fractions of Doc Len"))

x, y = zip(*sorted(span_width.items()))
fig.add_trace(
    go.Histogram(x=doc_lengths),
    row=1, col=1
)

fig.add_trace(
    go.Histogram(x=num_bert_windows),
    row=1, col=2
)

fig.add_trace(
    go.Bar(x=x, y=y, width=0.2),
    row=2, col=1
)

fig.add_trace(
    go.Histogram(x=span_count_fracs),
    row=2, col=2
)

fig.update_layout(showlegend=False)
fig.show()